In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import Recognition
import KNN
from PIL import Image
import change0_1
import solve

In [2]:
import requests
import random
url='http://cvapi.gdieee.com/get_sudoku'
params={
    'id':random.randint(1,1000)
}
r=requests.get(url,params=params)  #params的参数必须为字典
fr=open('/home/mai/photo','wb')    #open打开的是文档，不是文件夹
fr.write(r.content)   #r.content为内容
fr.close()

In [3]:
img = cv2.imread('photo')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
## 阈值分割
ret,thresh = cv2.threshold(gray,200,255,1)

##膨胀
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5, 5))     
dilated = cv2.dilate(thresh,kernel)
 
## 轮廓提取（findContours返回contours(组成轮廓的点集)，hierarchy(层级结构)）
contours, hierarchy = cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)  #cv2.RETR_TREE提取所有轮廓，建立网状的轮廓结构

##　提取八十一个小方格
boxes = []
for i in range(len(hierarchy[0])):
    if hierarchy[0][i][3] == 0:         #父轮廓都为0号轮廓（hierarchy[Next][Previous][Child][Parant])
        boxes.append(hierarchy[0][i])
        
height,width = img.shape[:2]       #img.shape[:2] 取彩色图片的长、宽，如果img.shape[:3] 取取彩色图片的长、宽、通道 
box_h = height/9
box_w = width/9
#print("box_h:",box_h)
#print("box_w:",box_w)
number_boxes = []

## 数独初始化为零阵
soduko = np.zeros((9, 9),np.int32)
count=0

for j in range(len(boxes)):
    if boxes[j][2] != -1:              #存在子轮廓
        x,y,w,h = cv2.boundingRect(contours[boxes[j][2]])    #左上角为（0，0）坐标系，起始x,y点，w宽，h高
        #print(x,y,w,h)
        number_boxes.append([x,y,w,h])
        #img = cv2.rectangle(img,(x-1,y-1),(x+w+1,y+h+1),(0,0,255),2)
        #plt.imshow(img)
        
        ## 统一大小（cv2.resize（img，（宽，高））
        resized_roi=cv2.resize(gray[y:y+h, x:x+w],(200,200))
        #保存图片
        cv2.imwrite('/home/mai/imgFile/%s.pgm'%str(count),resized_roi)
        
        #转化为0—1图片
        imgDir='/home/mai/imgFile'
        change0_1.changeImgFile(imgDir,count)

        #数字识别
        traindir= '/home/mai/test/mytrain'        #训练集提取了当前api端口图片1-9数字0-1图像，数量少但准确率100%，使用于当前题目
        testdir = '/home/mai/test/mydigits'
        number=Recognition.handwriteClassfiy(testdir, traindir, 1, count)
        count+=1
        
        ## 识别结果展示（cv2.putText（图片，添加的文字，角坐标，字体，字体大小，颜色，字体粗细））
        cv2.putText(img,str(number),(x+w,y+h-20), 3, 1., (255, 0, 0),2)
        
        ## 求在矩阵中的位置，写入数字
        #print("行：",int(y/box_h))
        #print("列：",int(x/box_w))
        if y<=227:                #当y<=227时，所得函数比实际值大1（原因不明，输出x,y,w,h和行列数后可看出错误）仅适用于从当前api口获得的图片
            soduko[int(y/box_h-1)][int(x/box_w)]=number
        else:
            soduko[int(y/box_h)][int(x/box_w)]=number

cv2.namedWindow("img", cv2.WINDOW_NORMAL);
cv2.imshow("img", img)
cv2.waitKey(30)
print("\n生成的数独\n")
print(soduko)


error: OpenCV(4.0.0) /io/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [1]:
print("\n求解后的数独\n")

## 数独求解
solve.solveSudoku(soduko)

print(soduko)
print("\n验算：求每行每列的和\n")
row_sum = map(sum,soduko)
col_sum = map(sum,zip(*soduko))
print(list(row_sum))
print(list(col_sum))

#print(sum(soduko.transpose))
## 把结果按照位置填入图片中  
for i in range(9):
    for j in range(9):
        x = int((i+0.25)*box_w)
        y = int((j+0.5)*box_h)
        cv2.putText(img,str(soduko[j][i]),(x,y), 3, 2., (0, 0, 255), 2, cv2.LINE_AA)

cv2.namedWindow("img", cv2.WINDOW_NORMAL)
cv2.imshow("img", img)
cv2.waitKey(30)


求解后的数独



NameError: name 'solve' is not defined